### Import Dependencies

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, MatchAny, FieldCondition, Filter, Prefetch, FusionQuery

import pandas as pd
import numpy as np
import openai
import json
import tiktoken

#### Retrieve all item IDs from Amazon Items Qdrant Collection

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
dummy_vector = np.zeros(1536).tolist()

In [4]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [5]:
payload.points

[ScoredPoint(id=201, version=3, score=0.0, payload={'parent_asin': 'B09SBZZVL3'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=429, version=3, score=0.0, payload={'parent_asin': 'B0B2JYLGNX'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=374, version=3, score=0.0, payload={'parent_asin': 'B09YCSP9FH'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=473, version=3, score=0.0, payload={'parent_asin': 'B0C1YLPFRK'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=407, version=3, score=0.0, payload={'parent_asin': 'B0C89R9NFZ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=97, version=3, score=0.0, payload={'parent_asin': 'B09YCXVRFN'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=82, version=3, score=0.0, payload={'parent_asin': 'B0BCGGV8R1'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=95, version=3, score=0.0, payload={'parent_asin': 'B0B2DNMM8Y'}, vector=None

In [6]:
len(payload.points)

1000

In [7]:
parent_asin_list = [item.payload["parent_asin"] for item in payload.points]

In [8]:
parent_asin_list

['B09SBZZVL3',
 'B0B2JYLGNX',
 'B09YCSP9FH',
 'B0C1YLPFRK',
 'B0C89R9NFZ',
 'B09YCXVRFN',
 'B0BCGGV8R1',
 'B0B2DNMM8Y',
 'B0B96RSG2Y',
 'B09TZZR8DB',
 'B09SVN1YLY',
 'B0BKFPSBDQ',
 'B07MZRWKWM',
 'B0B9RXGWD5',
 'B0BMKH3KYC',
 'B0C7WKF7T6',
 'B0BVDGRLDZ',
 'B0BMLV9V7V',
 'B0B764M1MF',
 'B0B8NK3TPY',
 'B0BD9DVMCX',
 'B0BRY7T37H',
 'B09ZP22RN4',
 'B0B54FNTZ3',
 'B0BML3SQSM',
 'B0B6CHRTGJ',
 'B09XF1JXXB',
 'B0BVM12TMZ',
 'B0B8N2ZXX4',
 'B0BZ871HQC',
 'B0BC19YPZG',
 'B0BJKPTD7J',
 'B0BP1RZSF4',
 'B0BDLVB11Y',
 'B08CXFGV6B',
 'B0B8G12853',
 'B0BJ5ZCY2V',
 'B0BV6GY5LD',
 'B09QRR4C8Y',
 'B0BXNZY997',
 'B0BRSMHRS3',
 'B0BVB5WXGV',
 'B09TSRX7LD',
 'B0B57S52FS',
 'B0B3N3P3P1',
 'B09Y1FVYNW',
 'B0B3DCLY8H',
 'B0B15NNYSR',
 'B0C4FDPB28',
 'B0B5W8GLYG',
 'B0BRXLKBW1',
 'B0C77XMZ4K',
 'B0BGMCHZVL',
 'B0B5H3NW22',
 'B09MQJ1KZQ',
 'B0CG1C78YF',
 'B0B97LFT8Q',
 'B0BM5RCRHK',
 'B09TTVBJZV',
 'B0BBGMCDTP',
 'B09ZY72MQ1',
 'B0B9MZPGVD',
 'B0B2R74DGX',
 'B0BPBTWX5Z',
 'B0BWMJ53MD',
 'B0C4DLHNRD',
 'B0CCKQ9H

#### Load Amazon Reviews Dataset

In [9]:
df_reviews = pd.read_json("../../data/Video_Games_2022_2023_with_category_ratings_10_sample_1000.jsonl", lines=True)

In [10]:
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,The Only One You Need!,My 21 year old son is picky about his headphon...,[],B07V5DJY4D,B0B3XNR8YM,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2019-10-27 21:28:43.283,0,False
1,5,Good product good price,Would buy again,[],B09BNBMT8Y,B0BRF6PX4Z,AFVP66AZENGK2SKTLADNPJGIYPTQ,2022-02-28 01:13:46.082,0,True
2,5,Great for Fortnite,Got this for using with Fornight and also doin...,[],B08RDGGJVV,B08RDGFR94,AFJBKPK5W56XWSNPQU2WW66ISWYQ,2021-05-19 19:30:57.012,0,False
3,5,"Great value! I'm not a bot! Really, I'm a hu...",My 11 yr old son just received this set with h...,[],B07RHRMBVB,B0BPBTWX5Z,AHBI5SLZDP3Q3LZPETJLCHQFGLUA,2022-12-16 20:47:00.997,0,True
4,1,Dead from the start- would NOT even pair to PS...,It pains me to say… PS4 controller left us dea...,[],B09QCL2RMX,B09QCL2RMX,AEIPJBAN7A55Q5DFFPZSR2UV3OKA,2022-07-30 00:30:28.616,0,False


In [11]:
len(df_reviews)

38124

In [12]:
df_reviews_sample = df_reviews[df_reviews["parent_asin"].isin(parent_asin_list)]

In [13]:
len(df_reviews_sample)

38124

#### Define functions to preprocess reviews data

In [14]:
def preprocess_reviews_data(row):
    return f"{row['title']} {row['text']}"

In [15]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

In [16]:
encoding.encode("Can I get some earphones")

[6854, 358, 636, 1063, 2487, 17144]

In [17]:
def token_count(row, model="text-embedding-3-small"):

    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(row["preprocessed_data"]))

In [18]:
df_reviews_sample["preprocessed_data"] = df_reviews_sample.apply(preprocess_reviews_data, axis=1)

In [19]:
df_reviews_sample["preprocessed_data_token_count"] = df_reviews_sample.apply(token_count, axis=1)

In [20]:
df_reviews_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,preprocessed_data,preprocessed_data_token_count
0,5,The Only One You Need!,My 21 year old son is picky about his headphon...,[],B07V5DJY4D,B0B3XNR8YM,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2019-10-27 21:28:43.283,0,False,The Only One You Need! My 21 year old son is p...,137
1,5,Good product good price,Would buy again,[],B09BNBMT8Y,B0BRF6PX4Z,AFVP66AZENGK2SKTLADNPJGIYPTQ,2022-02-28 01:13:46.082,0,True,Good product good price Would buy again,7
2,5,Great for Fortnite,Got this for using with Fornight and also doin...,[],B08RDGGJVV,B08RDGFR94,AFJBKPK5W56XWSNPQU2WW66ISWYQ,2021-05-19 19:30:57.012,0,False,Great for Fortnite Got this for using with For...,69
3,5,"Great value! I'm not a bot! Really, I'm a hu...",My 11 yr old son just received this set with h...,[],B07RHRMBVB,B0BPBTWX5Z,AHBI5SLZDP3Q3LZPETJLCHQFGLUA,2022-12-16 20:47:00.997,0,True,"Great value! I'm not a bot! Really, I'm a hu...",121
4,1,Dead from the start- would NOT even pair to PS...,It pains me to say… PS4 controller left us dea...,[],B09QCL2RMX,B09QCL2RMX,AEIPJBAN7A55Q5DFFPZSR2UV3OKA,2022-07-30 00:30:28.616,0,False,Dead from the start- would NOT even pair to PS...,214


In [21]:
len(df_reviews_sample)

38124

In [22]:
df_reviews_sample = df_reviews_sample[df_reviews_sample["preprocessed_data_token_count"] < 8192]

In [23]:
len(df_reviews_sample)

38124

In [24]:
total_tokens = df_reviews_sample["preprocessed_data_token_count"].sum()

In [25]:
total_tokens

np.int64(2642481)

#### Create a new Qdrant collection for Reviews

In [26]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-01-reviews",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [27]:
qdrant_client.create_payload_index(
    collection_name="Amazon-items-collection-01-reviews",
    field_name="parent_asin",
    field_schema=PayloadSchemaType.KEYWORD
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

#### Embedding Functions

In [28]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [29]:
def get_embeddings_batch(text_list, model="text-embedding-3-small", batch_size=100):
    
    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input=text_list, model=model)
        return [embedding.embedding for embedding in response.data]
    
    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed {counter * batch_size} of {len(text_list)}")
        counter += 1
    
    return all_embeddings

#### Embed the text and add additional fields to the payload of each vector for reviews

In [30]:
data_to_embed_reviews = df_reviews_sample[["preprocessed_data", "parent_asin"]].to_dict(orient="records")

In [31]:
data_to_embed_reviews

[{'preprocessed_data': 'The Only One You Need! My 21 year old son is picky about his headphones. First of all, he has a large head and finds some to be uncomfortable. This one easily expands to fit his head (and his hair! He has a lot!)  The ear cups are padded well and cover his ears completely. Another thing he is picky about is the bass. He likes a lot of bass and finds these to be acceptable. He really likes that he can use these with his computer and his phone. He can even use it with his gaming systems. One final thing, he likes they way they look. They have a youthful appearance compared to many out there.',
  'parent_asin': 'B0B3XNR8YM'},
 {'preprocessed_data': 'Good product good price Would buy again',
  'parent_asin': 'B0BRF6PX4Z'},
 {'preprocessed_data': "Great for Fortnite Got this for using with Fornight and also doing general media editing with Adobe CS apps. Great DPI, just as good if not better than my wired mouse. Excellent charged time, easy makes it to 40+ hours on o

In [32]:
text_to_embed_reviews = [data["preprocessed_data"] for data in data_to_embed_reviews]

In [33]:
text_to_embed_reviews

['The Only One You Need! My 21 year old son is picky about his headphones. First of all, he has a large head and finds some to be uncomfortable. This one easily expands to fit his head (and his hair! He has a lot!)  The ear cups are padded well and cover his ears completely. Another thing he is picky about is the bass. He likes a lot of bass and finds these to be acceptable. He really likes that he can use these with his computer and his phone. He can even use it with his gaming systems. One final thing, he likes they way they look. They have a youthful appearance compared to many out there.',
 'Good product good price Would buy again',
 "Great for Fortnite Got this for using with Fornight and also doing general media editing with Adobe CS apps. Great DPI, just as good if not better than my wired mouse. Excellent charged time, easy makes it to 40+ hours on one charges. Lights give it a great look that's not too obnoxious. Overall very pleased.",
 'Great value!  I\'m not a bot!  Really,

In [34]:
embeddings_reviews = get_embeddings_batch(text_to_embed_reviews, batch_size=500)

Processed 500 of 38124
Processed 1000 of 38124
Processed 1500 of 38124
Processed 2000 of 38124
Processed 2500 of 38124
Processed 3000 of 38124
Processed 3500 of 38124
Processed 4000 of 38124
Processed 4500 of 38124
Processed 5000 of 38124
Processed 5500 of 38124
Processed 6000 of 38124
Processed 6500 of 38124
Processed 7000 of 38124
Processed 7500 of 38124
Processed 8000 of 38124
Processed 8500 of 38124
Processed 9000 of 38124
Processed 9500 of 38124
Processed 10000 of 38124
Processed 10500 of 38124
Processed 11000 of 38124
Processed 11500 of 38124
Processed 12000 of 38124
Processed 12500 of 38124
Processed 13000 of 38124
Processed 13500 of 38124
Processed 14000 of 38124
Processed 14500 of 38124
Processed 15000 of 38124
Processed 15500 of 38124
Processed 16000 of 38124
Processed 16500 of 38124
Processed 17000 of 38124
Processed 17500 of 38124
Processed 18000 of 38124
Processed 18500 of 38124
Processed 19000 of 38124
Processed 19500 of 38124
Processed 20000 of 38124
Processed 20500 of 3

In [35]:
len(embeddings_reviews)

38124

In [36]:
pointstructs = []
i = 1
for embedding, data in zip(embeddings_reviews, data_to_embed_reviews):
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload={
                "text": data["preprocessed_data"],
                "parent_asin": data["parent_asin"],
            }
        )
    )
    i += 1

In [37]:
batch_size_qdrant = 100
counter = 1
for i in range(0, len(pointstructs), batch_size_qdrant):
    batch = pointstructs[i:i + batch_size_qdrant]
    qdrant_client.upsert(
        collection_name="Amazon-items-collection-01-reviews",
        wait=True,
        points=batch
    )
    print(f"Processed {counter * batch_size_qdrant} of {len(pointstructs)}")
    counter += 1

Processed 100 of 38124
Processed 200 of 38124
Processed 300 of 38124
Processed 400 of 38124
Processed 500 of 38124
Processed 600 of 38124
Processed 700 of 38124
Processed 800 of 38124
Processed 900 of 38124
Processed 1000 of 38124
Processed 1100 of 38124
Processed 1200 of 38124
Processed 1300 of 38124
Processed 1400 of 38124
Processed 1500 of 38124
Processed 1600 of 38124
Processed 1700 of 38124
Processed 1800 of 38124
Processed 1900 of 38124
Processed 2000 of 38124
Processed 2100 of 38124
Processed 2200 of 38124
Processed 2300 of 38124
Processed 2400 of 38124
Processed 2500 of 38124
Processed 2600 of 38124
Processed 2700 of 38124
Processed 2800 of 38124
Processed 2900 of 38124
Processed 3000 of 38124
Processed 3100 of 38124
Processed 3200 of 38124
Processed 3300 of 38124
Processed 3400 of 38124
Processed 3500 of 38124
Processed 3600 of 38124
Processed 3700 of 38124
Processed 3800 of 38124
Processed 3900 of 38124
Processed 4000 of 38124
Processed 4100 of 38124
Processed 4200 of 38124
P

#### A function to run search against reviews on a prefiltered set of product IDs

In [38]:
def retrieve_prefiltered_reviews_data(query, parent_asins, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-reviews",
        prefetch=[
            Prefetch(
                query=query_embedding,
                filter=Filter(
                    must=[
                        FieldCondition(
                            key="parent_asin",
                            match=MatchAny(
                                any=parent_asins
                            )
                        )
                    ]
                ),
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )

    return results

In [45]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B0BMWVMBLJ"], k=5)

In [46]:
reviews.points

[ScoredPoint(id=10343, version=106, score=0.5, payload={'text': 'Poor quality The little man broke off before we could even put batteries in it.', 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=30392, version=306, score=0.33333334, payload={'text': 'Hard to work Had a hard time getting these things to recharge right and one of their heads came off in a few days and is unfixable. Served the purpose though. I really wasn’t expecting much.', 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=34385, version=346, score=0.25, payload={'text': "#BONELESS review. Hello Amazon, this toy is NO FUN! It doesn't work right. I HATE THS TOY!<br /><br />Amazon people for your in for information THIS IS A PIECE OF TRASH!!!!!!!!!!", 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12643, version=129, score=0.2, payload={'text': "Broke within 5 min. I bought this as a gif

In [47]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B0BMWVMBLJ", "B09QY98TXK"], k=5)

In [48]:
reviews.points

[ScoredPoint(id=10343, version=106, score=0.5, payload={'text': 'Poor quality The little man broke off before we could even put batteries in it.', 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=30392, version=306, score=0.33333334, payload={'text': 'Hard to work Had a hard time getting these things to recharge right and one of their heads came off in a few days and is unfixable. Served the purpose though. I really wasn’t expecting much.', 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=34385, version=346, score=0.25, payload={'text': "#BONELESS review. Hello Amazon, this toy is NO FUN! It doesn't work right. I HATE THS TOY!<br /><br />Amazon people for your in for information THIS IS A PIECE OF TRASH!!!!!!!!!!", 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12643, version=129, score=0.2, payload={'text': "Broke within 5 min. I bought this as a gif

In [49]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B0BMWVMBLJ", "B09QY98TXK"], k=20)

In [50]:
reviews.points

[ScoredPoint(id=10343, version=106, score=0.5, payload={'text': 'Poor quality The little man broke off before we could even put batteries in it.', 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=30392, version=306, score=0.33333334, payload={'text': 'Hard to work Had a hard time getting these things to recharge right and one of their heads came off in a few days and is unfixable. Served the purpose though. I really wasn’t expecting much.', 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=34385, version=346, score=0.25, payload={'text': "#BONELESS review. Hello Amazon, this toy is NO FUN! It doesn't work right. I HATE THS TOY!<br /><br />Amazon people for your in for information THIS IS A PIECE OF TRASH!!!!!!!!!!", 'parent_asin': 'B0BMWVMBLJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12643, version=129, score=0.2, payload={'text': "Broke within 5 min. I bought this as a gif

In [51]:
for point in reviews.points:
    print(point.payload["parent_asin"])
    print(point.payload["text"])
    print("-"*100)

B0BMWVMBLJ
Poor quality The little man broke off before we could even put batteries in it.
----------------------------------------------------------------------------------------------------
B0BMWVMBLJ
Hard to work Had a hard time getting these things to recharge right and one of their heads came off in a few days and is unfixable. Served the purpose though. I really wasn’t expecting much.
----------------------------------------------------------------------------------------------------
B0BMWVMBLJ
#BONELESS review. Hello Amazon, this toy is NO FUN! It doesn't work right. I HATE THS TOY!<br /><br />Amazon people for your in for information THIS IS A PIECE OF TRASH!!!!!!!!!!
----------------------------------------------------------------------------------------------------
B0BMWVMBLJ
Broke within 5 min. I bought this as a gift and the peg the guy stands on the skateboard broke off in his foot.  It also isn't very fun. Doesn't do any of the flips and tricks it supposedly does. As soon